In [7]:
import tensorflow as tf
import bert
import tensorflow_hub as hub

In [8]:
model = tf.keras.models.load_model('models/text_model_1')
model.summary()

Model: "text_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_220 (Embedding)    multiple                  7935720   
_________________________________________________________________
conv1d_660 (Conv1D)          multiple                  26050     
_________________________________________________________________
conv1d_661 (Conv1D)          multiple                  39050     
_________________________________________________________________
conv1d_662 (Conv1D)          multiple                  52050     
_________________________________________________________________
global_max_pooling1d_220 (Gl multiple                  0         
_________________________________________________________________
dense_440 (Dense)            multiple                  38656     
_________________________________________________________________
dropout_220 (Dropout)        multiple                  0

In [9]:
#make pre-train for bert
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [10]:
def tokenize_data(data):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(data))

In [14]:
s = """
    I am trying to create a multi label text classification model in tensorflow. I was not sure if I should use
    Tensorflow or PyTorch, I decided to use Tensorflow because I am more familiar with it. I am testing this model
    with the freecode dataset and TagBERT. Is this the best model for my use case?
    """

the_tokens = tokenize_data(s)
the_tokens

[1045,
 2572,
 2667,
 2000,
 3443,
 1037,
 4800,
 3830,
 3793,
 5579,
 2944,
 1999,
 23435,
 12314,
 1012,
 1045,
 2001,
 2025,
 2469,
 2065,
 1045,
 2323,
 2224,
 23435,
 12314,
 2030,
 1052,
 22123,
 2953,
 2818,
 1010,
 1045,
 2787,
 2000,
 2224,
 23435,
 12314,
 2138,
 1045,
 2572,
 2062,
 5220,
 2007,
 2009,
 1012,
 1045,
 2572,
 5604,
 2023,
 2944,
 2007,
 1996,
 2489,
 16044,
 2951,
 13462,
 1998,
 6415,
 8296,
 1012,
 2003,
 2023,
 1996,
 2190,
 2944,
 2005,
 2026,
 2224,
 2553,
 1029]

In [18]:
PREDICTION_BATCH_SIZE = 512
chunk = [the_tokens]
processed_chunk_dataset = tf.data.Dataset.from_generator(lambda: chunk, output_types=(tf.int32))
batched_chunk_dataset = processed_chunk_dataset.padded_batch(PREDICTION_BATCH_SIZE, padded_shapes=(None,))

# rows = text_model.predict(batched_chunk_dataset, batch_size=len(chunk))
model.predict(batched_chunk_dataset)

2022-01-03 16:12:56.777186: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8204
2022-01-03 16:12:57.399009: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


array([[0.43726867]], dtype=float32)